In [1]:
! pip install --upgrade --quiet nltk utils indic-nlp-library

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 2.2 MB/s eta 0:00:00


In [2]:
import json
import nltk
import re
from indicnlp.tokenize import indic_tokenize
from nltk.translate.gleu_score import sentence_gleu, corpus_gleu

# GLEU

In [3]:
def WordTokenizer(data, corpus='prose', keep_punctuations=True, keep_stopwords = True):
    # print(indic_tokenize.trivial_tokenize(data, lang='gu'))
    # return indic_tokenize.trivial_tokenize(data, lang='gu')

    stopwords = [
      "અથવા", "અને", "અમને", "અમારું", "અમે", "અહીં", "આ", "આગળ", "આથી", "આનું", "આને", "આપણને", "આપણું",
      "આપણે", "આપી", "આવી", "આવે", "ઉપર", "ઊંચે", "ઊભું", "એ", "એક", "એના", "એનાં", "એની", "એનું", "એને",
      "એનો", "એમ", "એવા", "એવાં", "એવી", "એવું", "એવો", "ઓછું", "અંગે", "અંદર", "કઈ", "કયું", "કયો", "કરવું",
      "કરતાં", "કરી", "કરીએ", "કરું", "કરે", "કરેલું", "કર્યા", "કર્યાં", "કર્યું", "કર્યો", "કંઈક", "કાંઈ",
      "કે", "કેટલું", "કેમ", "કેવી", "કેવું", "કોઈ", "કોઈક", "કોણ", "કોણે", "કોને", "ક્યારે", "ક્યાં", "ખૂબ",
      "ગઈ", "ગયા", "ગયાં", "ગયું", "ગયો", "ઘણું", "છ", "છતાં", "છીએ", "છું", "છે", "છેક", "છો", "જ", "જાય",
      "જી", "જે", "જેટલું", "જેને", "જેમ", "જેવી", "જેવું", "જેવો", "જો", "જોઈએ", "જ્યારે", "જ્યાં", "ઝાઝું",
      "તને", "તમને", "તમારું", "તમે", "તારાથી", "તારામાં", "તારું", "તું", "તે", "તેઓ", "તેથી", "તેણે",
      "તેના", "તેની", "તેનું", "તેને", "તેમ", "તેમનું", "તેમને", "તેવી", "તેવું", "તેં", "તો", "ત્યારે", "ત્યાં",
      "થઈ", "થઈએ", "થતા", "થતાં", "થતી", "થતું", "થતો", "થયા", "થયાં", "થયું", "થયો", "થયેલું", "થવું", "થાઉં",
      "થાઓ", "થાય", "થોડું", "દરેક", "ન", "નથી", "નહિ", "નહીં", "નં.", "ના", "નીચે", "ને", "પછી", "પણ", "પર",
      "પરંતુ", "પહેલાં", "પાછળ", "પાસે", "પોતાનું", "પ્રત્યેક", "ફક્ત", "ફરી", "ફરીથી", "બધા", "બધું", "બની",
      "બહાર", "બહુ", "બંને", "બાદ", "બે", "મને", "મા", "માટે", "માત્ર", "મારું", "મૂકવું", "મૂકી", "મૂક્યા",
      "મૂક્યાં", "મૂક્યું", "મેં", "રહી", "રહે", "રહેવું", "રહ્યા", "રહ્યાં", "રહ્યો", "રીતે", "રૂ.", "લેતા",
      "લેતું", "લેવા", "વગેરે", "વધુ", "શકે", "શા", "શું", "સરખું", "સામે", "સુધી", "હતા", "હતાં", "હતી",
      "હતું", "હશે", "હશો", "હવે", "હા", "હું", "હો", "હોઈ", "હોઈશ", "હોઈશું", "હોય", "હોવા"
    ]

    if not keep_punctuations:
        data = re.sub(r'[.?/,:;*\"!$#@%^&~\(\)]','',data)

    data = re.sub(r'([.,\'\\"!?%#@*<>|\+\-\(\)])', r' \1', data)
    data = re.sub(r'[।।(૧૨૩૪૫૬૭૮૯)*।।]', '  ', data)
    data = re.sub(r"   ", '', data)
    data = re.sub(r'…', " ", data)
    data = re.split(r'[ -]',data)
    return_list = []
    if not keep_stopwords and corpus=='poetry':
        a = []
        for word in data:
          if word not in stopwords:
        	  a.append(word)
        return a

    if not keep_stopwords and corpus=='prose':
        a = []
        for word in data:
        	if word not in stopwords:
        		a.append(word)
        return a


    for i in data:
        if i:
            return_list.append(i)


    return return_list

def SentenceTokenizer(data):
    data = data.strip()
    data = re.sub(r'([.!?])', r'\1 ', data)
    data = re.split(r'  ',data)
    if not data[-1]:
    	del(data[-1])
    return data

In [30]:
def load_json_data(filepath):
    """Loads JSON data from a file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def calculate_avg_gleu(data):
    """Calculates average (sentence-level) GLEU."""
    total_gleu_score = 0
    num_sentences = 0

    for entry in data:
        predicted = entry["prediction"]
        correct_sentences = entry["reference"]
        # predicted = entry["input"]
        # correct_sentences = entry["prediction"]
        tokenized_predicted = WordTokenizer(predicted)
        tokenized_correct = [WordTokenizer(s) for s in correct_sentences]

        gleu_score = sentence_gleu(tokenized_correct, tokenized_predicted)
        total_gleu_score += gleu_score
        num_sentences += 1

    if num_sentences == 0:
        return 0

    return total_gleu_score / num_sentences

def calculate_corpus_gleu(data):
    """Calculates corpus-level GLEU."""
    references = []
    hypotheses = []

    for entry in data:
        predicted = entry["prediction"]
        correct_sentences = entry["reference"]
        # predicted = entry["input"]
        # correct_sentences = entry["prediction"]
        tokenized_predicted = WordTokenizer(predicted)
        tokenized_correct = [WordTokenizer(s) for s in correct_sentences]

        references.append(tokenized_correct)
        hypotheses.append(tokenized_predicted)

    return corpus_gleu(references, hypotheses)

# Main evaluation
try:
    # json_data = load_json_data("/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/evaluation_set.json")
    json_data = load_json_data("/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/evaluation_results/gpt-4o_cot_with_m1&m2_k1=5_k2=5.json")["results"]
    # print(json_data)
    overall_gleu = calculate_avg_gleu(json_data)
    corpus_gleu_score = calculate_corpus_gleu(json_data)

    print(f"Sentence-level GLEU score (average): {overall_gleu:.8f}")
    print(f"Corpus-level GLEU score: {corpus_gleu_score:.8f}")

except FileNotFoundError:
    print("Error: JSON file not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Sentence-level GLEU score (average): 0.73928271
Corpus-level GLEU score: 0.76210598


In [5]:
def calculate_sentence_gleu(predicted_sentence, correct_sentences):
  """Calculates GLEU for a single sentence pair.

  Args:
    predicted_sentence: The predicted sentence string.
    correct_sentences: A list of correct sentence strings.

  Returns:
    The sentence-level GLEU score.
  """
  tokenized_predicted = WordTokenizer(predicted_sentence)
  tokenized_correct = [WordTokenizer(s) for s in correct_sentences]
  return sentence_gleu(tokenized_correct, tokenized_predicted)

predicted = "મેં તેને સેક્રેટરી દ્વારા કહેવરવ્યું છે."
correct = ["મેં તેને સેક્રેટરી દ્વારા કહેવરાવ્યું છે.",
        "મેં તેને સેક્રેટરી દ્વારા કહેવડાવ્યું છે."]

gleu_score = calculate_sentence_gleu(predicted, correct)
print(f"Sentence GLEU score: {gleu_score:.4f}")

Sentence GLEU score: 0.5909


# M^2

In [8]:
!git clone https://github.com/keisks/m2scorer.git

Cloning into 'm2scorer'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 67 (delta 27), reused 24 (delta 24), pack-reused 32 (from 1)
Receiving objects: 100% (67/67), 36.25 KiB | 447.00 KiB/s, done.
Resolving deltas: 100% (33/33), done.


In [31]:
with open('parallel.txt', 'w', encoding='utf-8') as f:
    for entry in json_data:
        incorrect = entry["input"].strip()
        # incorrect = entry["incorrect_sentence"].strip()
        # correct_sentences = [entry["prediction"].strip()]
        correct_sentences = [s.strip() for s in entry["reference"]]
        line = incorrect + ' || ' + ' | '.join(correct_sentences) + '\n'
        f.write(line)

In [32]:
with open('pred.txt', 'w', encoding='utf-8') as f:
    for entry in json_data:
        predicted = entry["prediction"].strip()
        # predicted = entry["input"].strip()

        predicted = ' '.join(WordTokenizer(predicted))
        # correct_sentences = entry["prediction"]
        correct_sentences = entry["reference"]
        f.write(predicted + '\n')
print("Predicted values written to pred.txt")

Predicted values written to pred.txt


In [33]:
import difflib
from collections import defaultdict

def tokenize_gu(text):
    return WordTokenizer(text)

def generate_edits(src_tokens, ref_tokens, annotator_id):
    matcher = difflib.SequenceMatcher(None, src_tokens, ref_tokens)
    edits = []
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            continue
        corrections = ' '.join(ref_tokens[j1:j2]) if j1 < j2 else '-NONE-'
        err_type = {
            'replace': 'Replacement',
            'insert':  'Insertion',
            'delete':  'Deletion'
        }[tag]
        edits.append(f"A {i1} {i2}|||{err_type}|||{corrections}|||REQUIRED|||-NONE-|||{annotator_id}")
    return edits

def generate_m2_from_parallel(input_path, output_path="gold.m2"):
    sentence_blocks = []

    with open(input_path, 'r', encoding='utf-8') as f:
        lines = [l.strip() for l in f if l.strip()]

    for line in lines:
        src_text, refs_part = line.split('||', 1)
        src_tokens = tokenize_gu(src_text)
        refs = [r.strip() for r in refs_part.split('|') if r.strip()]

        s_line = f"S {' '.join(src_tokens)}"
        all_edits = []

        for ann_id, ref in enumerate(refs):
            ref_tokens = tokenize_gu(ref)
            edits = generate_edits(src_tokens, ref_tokens, annotator_id=str(ann_id))
            if not edits:
                all_edits.append(f"A -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||{ann_id}")
            else:
                all_edits.extend(edits)

        sentence_blocks.append((s_line, all_edits))

    # Write all at once
    with open(output_path, 'w', encoding='utf-8') as out:
        for s_line, edits in sentence_blocks:
            out.write(s_line + "\n")
            for edit in edits:
                out.write(edit + "\n")
            out.write("\n")

    print(f"Generated {output_path} successfully.")

In [34]:
print(len(json_data))

300


In [35]:
generate_m2_from_parallel("parallel.txt", "gold.m2")

Generated gold.m2 successfully.


In [29]:
# prompt: I have a text file containing one Gujarati sentence per line. Write code, which randomly chooses a sentence from the file and asks user. If user gives 'y' as input, it appends it to another text file and 'n' to skip. Loop until 'e' is encountered.

import random

def process_gujarati_sentences(input_file, output_file):
    """
    Reads Gujarati sentences from an input file, randomly presents them to the user,
    and appends selected sentences to an output file.

    Args:
        input_file (str): The path to the input text file.
        output_file (str): The path to the output text file.
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as f_in:
            sentences = f_in.readlines()
            sentences = [s.strip() for s in sentences if s.strip()] # Remove leading/trailing whitespace and empty lines
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the input file: {e}")
        return

    if not sentences:
        print("Input file is empty.")
        return

    while True:
        # Choose a random sentence
        random_sentence = random.choice(sentences)

        print(f"\nSentence: {random_sentence}")
        user_input = input("Keep sentence? (y/n/e): ").lower()

        if user_input == 'y':
            try:
                with open(output_file, 'a', encoding='utf-8') as f_out:
                    f_out.write(random_sentence + '\n')
                print("Sentence added to output file.")
            except Exception as e:
                print(f"An error occurred while writing to the output file: {e}")
        elif user_input == 'n':
            print("Sentence skipped.")
        elif user_input == 'e':
            print("Exiting.")
            break
        else:
            print("Invalid input. Please enter 'y', 'n', or 'e'.")

# Example usage:
# Assuming your input file is named 'gujarati_sentences.txt' and output file is 'selected_sentences.txt'
# Replace with your actual file names
input_filename = '/content/Gujarati_sentences_with_err.txt'  # Replace with your input filename
output_filename = '/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/data/Gujarati_sentences_with_err.txt' # Replace with your output filename

process_gujarati_sentences(input_filename, output_filename)

Error: Input file '/content/Gujarati_sentences_with_err.txt' not found.


In [ ]:
# prompt: Write a code to remove duplicate lines from a text file.

def remove_duplicate_lines(input_filepath, output_filepath):
  """
  Removes duplicate lines from a text file and writes the unique lines to a new file.

  Args:
    input_filepath (str): The path to the input text file.
    output_filepath (str): The path to the output text file.
  """
  try:
    with open(input_filepath, 'r', encoding='utf-8') as infile:
      lines = infile.readlines()

    unique_lines = []
    seen_lines = set()

    for line in lines:
      # Strip leading/trailing whitespace before checking for uniqueness
      stripped_line = line.strip()
      if stripped_line not in seen_lines:
        unique_lines.append(line) # Keep the original line including newline character
        seen_lines.add(stripped_line)

    with open(output_filepath, 'w', encoding='utf-8') as outfile:
      outfile.writelines(unique_lines)

    print(f"Duplicate lines removed. Unique lines written to '{output_filepath}'.")

  except FileNotFoundError:
    print(f"Error: Input file '{input_filepath}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage:
input_file_to_clean = '/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/data/Gujarati_sentences_with_err.txt'
output_file_cleaned =

remove_duplicate_lines(input_file_to_clean, output_file_cleaned)


Duplicate lines removed. Unique lines written to '/content/Gujarati_sentences_with_err_cleaned.txt'.


In [ ]:
!git clone https://github.com/chrisjbryant/errant.git

Cloning into 'errant'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 331 (delta 122), reused 101 (delta 101), pack-reused 176 (from 2)
Receiving objects: 100% (331/331), 1.14 MiB | 5.38 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [ ]:
!cd /content/errant/errant

In [ ]:
! python /content/errant/errant/commands/compare_m2.py -hyp /content/hyp.m2 -ref /content/ref.m2 -cat 3 -ds


====================== Span-Based Detection ======================
Category       TP       FP       FN       P        R        F0.5
Deletion       8        3        17       0.7273   0.32     0.5797
Insertion      21       5        38       0.8077   0.3559   0.6442
Replacement    207      71       184      0.7446   0.5294   0.6886

============ Span-Based Detection ============
TP	FP	FN	Prec	Rec	F0.5
236	79	239	0.7492	0.4968	0.6801



In [ ]:
# prompt: load 3 json files (same format as gpt-4o_cot.json). Print the entries which have different prediction fields for all 3 files.

# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Assuming the data is under a "results" key as in the previous example
    return data.get("results", data)

# Load the three JSON files
file1_path = '/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/evaluation_results/gpt-4o_zs.json' # Replace with your actual file path
file2_path = '/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/evaluation_results/gpt-4o_cot_with_m1_k=5.json' # Replace with your actual file path
file3_path = '/content/drive/MyDrive/Gujarati_Spelling_and_Grammar_Autocorrect/evaluation_results/gpt-4o_cot_with_m1&m2_k1=5_k2=1.json' # Replace with your actual file path

try:
    data1 = load_json_data(file1_path)
    data2 = load_json_data(file2_path)
    data3 = load_json_data(file3_path)

    # Create dictionaries for faster lookup by some unique identifier (e.g., 'input' or an 'id' field)
    # Assuming 'input' is a unique identifier for each entry
    data1_dict = {entry['input']: entry for entry in data1}
    data2_dict = {entry['input']: entry for entry in data2}
    data3_dict = {entry['input']: entry for entry in data3}

    print("Entries with different predictions across all three files:")
    i = 0
    # Iterate through entries in one file and check corresponding entries in others
    for input_text, entry1 in data1_dict.items():
        # Check if the same input exists in the other two files
        if input_text in data2_dict and input_text in data3_dict:
            entry2 = data2_dict[input_text]
            entry3 = data3_dict[input_text]

            # Compare the 'prediction' field
            if True or entry1.get('prediction') != entry2.get('prediction') and entry1.get('prediction') != entry3.get('prediction') and entry2.get('prediction') != entry3.get('prediction'):
                i += 1
                print(f"\nInput: {input_text}")
                print(f"File 1 Prediction: {entry1.get('prediction', 'N/A')}")
                print(f"File 2 Prediction: {entry2.get('prediction', 'N/A')}")
                print(f"File 3 Prediction: {entry3.get('prediction', 'N/A')}")
                print(entry1.get('reference'))
    print(i)
except FileNotFoundError as e:
    print(f"Error: One of the JSON files not found: {e}")
except KeyError as e:
    print(f"Error: Missing expected key in JSON data: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Entries with different predictions across all three files:

Input: તેણે પેહલીવાર ઉંબરે બહાર પગ મુક્યો.
File 1 Prediction: તેણે પહેલીવાર ઉંબરે બહાર પગ મૂક્યો.
File 2 Prediction: તેણે પહેલીવાર ઉંબરે બહાર પગ મૂક્યો.
File 3 Prediction: તેણે પહેલીવાર ઉંબરે બહાર પગ મૂક્યો.
['તેણે પહેલી વાર ઉંબરા બહાર પગ મૂક્યો.', 'તેણે પહેલીવાર ઉંબરા બહાર પગ મૂક્યો.']

Input: દેવચકલીએ ચાર ચક્કર માર્યો.
File 1 Prediction: દેવચકલીએ ચાર ચક્કર માર્યા.
File 2 Prediction: દેવચકલીએ ચાર ચક્કર માર્યા.
File 3 Prediction: દેવચકલીએ ચાર ચક્કર માર્યા.
['દેવચકલીએ ચાર ચક્કર માર્યા.']

Input: ગામનાં લૉકૉ ભરમાયા છે.
File 1 Prediction: ગામનાં લોકો ભરમાયા છે.
File 2 Prediction: ગામનાં લોકો ભરમાયા છે.
File 3 Prediction: ગામનાં લોકો ભરમાયા છે.
['ગામનાં લોકો ભરમાયાં છે.']

Input: દેવચકલીએ ચાર ચક્કર માર્યાં.
File 1 Prediction: દેવચકલીએ ચાર ચક્કર માર્યા.
File 2 Prediction: દેવચકલીએ ચાર ચક્કર માર્યા.
File 3 Prediction: દેવચકલીએ ચાર ચક્કર માર્યાં.
['દેવચકલીએ ચાર ચક્કર માર્યા.']

Input: હું અત્યારે સાયકલ ચલાવી રહ્યો હતો.
File 1 Predict